# Default values for import

In [2]:
import json
import pickle
import requests
import time

url_matches_19_20 = 'https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%2C%0A((player_matches.player_slot%20%3C%20128)%20%3D%20matches.radiant_win)%20win%2C%0Aplayer_matches.hero_id%2C%0Aplayer_matches.account_id%2C%0Aleagues.name%20leaguename%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0AJOIN%20leagues%20using(leagueid)%0AJOIN%20player_matches%20using(match_id)%0AJOIN%20heroes%20on%20heroes.id%20%3D%20player_matches.hero_id%0ALEFT%20JOIN%20notable_players%20ON%20notable_players.account_id%20%3D%20player_matches.account_id%0ALEFT%20JOIN%20teams%20using(team_id)%0AWHERE%20TRUE%0AAND%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272019-12-31T23%3A00%3A00.000Z%27)%0AAND%20matches.start_time%20%3C%3D%20extract(epoch%20from%20timestamp%20%272020-12-31T23%3A00%3A00.000Z%27)%0AAND%20leagues.tier%20%3D%20%27professional%27%0AORDER%20BY%20matches.match_id%20NULLS%20LAST%0ALIMIT%20999999'
url_base_match = 'https://api.opendota.com/api/matches/'

class DotaTeam:
    def __init__(self,name,wins,losses,winrate,skill,real,luck):
        self.name = name
        self.wins = wins
        self.losses = losses
        self.winrate = winrate
        self.skill = skill
        self.real = real
        self.luck = luck
    def __str__(self):
        return str(self.__dict__)
    def __repr__(self):
        return str(self)
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__,
            sort_keys=True, indent=4)

class DotaMatch:
    def __init__(self,match_id,radiant_score,dire_score,duration,first_blood_time,picks_bans,radiant_team,dire_team):
        self.match_id = match_id
        self.radiant_score = radiant_score
        self.dire_score = dire_score
        self.duration = duration
        self.first_blood_time = first_blood_time
        self.pick_bans = picks_bans
        self.radiant_team = radiant_team
        self.dire_team = dire_team
    def __str__(self):
        return str(self.__dict__)
    def __repr__(self):
        return str(self)
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__,
            sort_keys=True, indent=4)

def save_obj(obj, name):
    with open('./Data/' + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


# Data collection for all Dota2 matches from 2019-2020 from the opendota API

In [3]:
# Data Generation
r_all_matches_19_20 = requests.get(url_matches_19_20)

In [4]:
parsed_json_matches_19_20 = json.loads(r_all_matches_19_20.text)
list_matches_19_20 = []

In [ ]:
for match in parsed_json_matches_19_20['rows']:
    if match['match_id'] not in list_matches_19_20:
        list_matches_19_20.append(match['match_id'])
print('Number of Matches: ', len(list_matches_19_20))

In [ ]:
fetch_counter = 1
matchdata_19_20 = []
for match in list_matches_19_20:
    matchdata_response = requests.get(url_base_match + str(match))
    if matchdata_response.status_code == 200:
        matchdata_19_20.append(json.loads(matchdata_response.text))
        if fetch_counter % 100 == 0:
            print('Request count: ', fetch_counter)
        fetch_counter +=1
        time.sleep(1.05)

In [ ]:
save_obj(matchdata_19_20, 'matchdata_19_20')